# Coursework 1: ML basics and fully-connected networks

#### Instructions

Please submit a version of this notebook containing your answers on CATe as *CW1*. Write your answers in the cells below each question.

We recommend that you work on the Ubuntu workstations in the lab. This assignment and all code were only tested to work on these machines. In particular, we cannot guarantee compatibility with Windows machines and cannot promise support if you choose to work on a Windows machine.

You can work from home and use the lab workstations via ssh (for list of machines: https://www.doc.ic.ac.uk/csg/facilities/lab/workstations). 

Once logged in, run the following commands in the terminal to set up a Python environment with all the packages you will need.

    export PYTHONUSERBASE=/vol/bitbucket/nuric/pypi
    export PATH=/vol/bitbucket/nuric/pypi/bin:$PATH

Add the above lines to your `.bashrc` to have these enviroment variables set automatically each time you open your bash terminal.

Any code that you submit will be expected to run in this environment. Marks will be deducted for code that fails to run.

Run `jupyter-notebook` in the coursework directory to launch Jupyter notebook in your default browser.

DO NOT attempt to create a virtualenv in your home folder as you will likely exceed your file quota.

**DEADLINE: 7pm, Tuesday 5th February, 2019**

## Part 1

1. Describe two practical methods used to estimate a supervised learning model's performance on unseen data. Which strategy is most commonly used in most deep learning applications, and why?
2. Suppose that you have reason to believe that your multi-layer fully-connected neural network is overfitting. List four things that you could try to improve generalization performance.

1. To estimate a supervised learning model's performance on unseen data, we can use either the mean squared error or the cross-entropy between the training data and the model distribution, as the cost function. The mean squared error enables to have the average output value for each observation. The cross-entropy is the negative log-likelihood and has better results in deep learning applications. Indeed, for gradient-based algorithms, the saturation of some outputs leads to small gradient values and hence to mediocre results.

2. To improve generalisation performance, we could try:
    - Regularisation with a norm penalty: by choosing a parameter norm penalty for the weights (e.g., $L^2$ regularisation), we can obtain effects such as more weights close to zero.
    -  Multi-task learning: by pooling examples, we create several tasks with shared parameters, which can also be seen as a kind of constraints on the parameters, thus leading to a better generalisation.
    - Early stopping: the training error always decreases with time and the test error increases after a certain threshold, which causes overfitting. By stopping the training earlier, we can hope to have a lower test error.
    - Bagging: by training several models and make them vote on the test set, we can try to have a better generalisation performance, since different models will not make the same errors.

## Part 2

1. Why can gradient-based learning be difficult when using the sigmoid or hyperbolic tangent functions as hidden unit activation functions in deep, fully-connected neural networks?
2. Why is the issue that arises in the previous question less of an issue when using such functions as output unit activation functions, provided that an appropriate loss function is used?
3. What would happen if you initialize all the weights to zero in a multi-layer fully-connected neural network and attempt to train your model using gradient descent? What would happen if you did the same thing for a logistic regression model?

1. The sigmoid and the hyperbolic tangent are related with $tanh(z) = 2 \sigma(2z) − 1$. Since the sigmoid function saturates on a wide domain, gradient-based learning is thus difficult with those functions. It is the vanishing gradient problem.

2. If the loss function can remove the saturation in the output layer, the previous problem is less relevant.

3. For a multi-layer fully-connected neural network, if all the weights are set to zero (and hence all the same value), the neural network will calculate the same function across the layers. Indeed, we will then have the same weights over the whole neural network, which is not useful. That is why for neural networks, we use a random distribution of weights. Conversely, for a logistic regression model, there is no such problem; a weight of zero means that the sigmoid function is in the linear area, which is convenient.


## Part 3

In this part, you will use PyTorch to implement and train a multinomial logistic regression model to classify MNIST digits.

Restrictions:
* You must use (but not modify) the code provided in `utils.py`. **This file is deliberately not documented**; read it carefully as you will need to understand what it does to complete the tasks.
* You are NOT allowed to use the `torch.nn` module.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultinomialLogisticRegressionClassifier` class below by filling in the missing parts (expected behaviour is prescribed in the documentation):
    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 92% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.


In [430]:
from utils import *

In [597]:
# *CODE FOR PART 3.1 IN THIS CELL*

class MultinomialLogisticRegressionClassifier:
    def __init__(self, weight_init_sd=100.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.weight_init_sd = weight_init_sd

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        self.variables = 28*28
        self.outcomes = 10
        
        m = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([self.weight_init_sd]))
        self.weights = m.sample(torch.Size([self.variables, self.outcomes])).view([self.variables, self.outcomes])
        
        # Bias added to the parameters
        self.weights = torch.cat((self.weights, torch.ones(1, self.outcomes)), 0)
        
        self.weights.requires_grad_()
        
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be a Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        flat_inputs = inputs.view(- 1, self.variables)
        batch_size = flat_inputs.shape[0]
        flat_bias = torch.cat((flat_inputs, torch.ones(batch_size, 1)), 1)
        matrix_mul = flat_bias.matmul(self.weights) # Parameters multiplied by the data
        
        matrix_bias_max = matrix_mul.max(dim = 1)[0].view([-1, 1])
        matrix_central = matrix_mul - matrix_bias_max
        
        # Softmax function
        exp_matrix_bias = torch.exp(matrix_central)
        row_exp_sum = exp_matrix_bias.sum(dim = 1).view([- 1, 1])
        log_normalised_row_exp = matrix_central - torch.log(row_exp_sum)
        return log_normalised_row_exp 
        
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return [self.weights]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        l1_weight = abs(self.weights).sum()
        return l1_weight
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        weights_prod = self.weights.matmul(self.weights.t())
        return torch.diag(weights_prod).sum() 
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [598]:
# *CODE FOR PART 3.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
torch.manual_seed(0)
model = MultinomialLogisticRegressionClassifier(weight_init_sd = 10.0)

res = run_experiment(
    model,
    optimizer = optim.Adam(model.parameters(), lr = 0.1),
    train_loader = train_loader_0,
    val_loader = val_loader_0,
    test_loader = test_loader_0,
    n_epochs = 9,
    l1_penalty_coef = 7.0,
    l2_penalty_coef = 7.0,
    suppress_output = False
)

Epoch 0: training...
Train set:	Average loss: 26.0925, Accuracy: 0.1157
Validation set:	Average loss: 2.2637, Accuracy: 0.1428

Epoch 1: training...
Train set:	Average loss: 2.2539, Accuracy: 0.2235
Validation set:	Average loss: 2.2395, Accuracy: 0.2858

Epoch 2: training...
Train set:	Average loss: 2.2456, Accuracy: 0.2333
Validation set:	Average loss: 2.2298, Accuracy: 0.2142

Epoch 3: training...
Train set:	Average loss: 2.2410, Accuracy: 0.2323
Validation set:	Average loss: 2.2578, Accuracy: 0.1888

Epoch 4: training...
Train set:	Average loss: 2.2399, Accuracy: 0.2235
Validation set:	Average loss: 2.2212, Accuracy: 0.2058

Epoch 5: training...
Train set:	Average loss: 2.2388, Accuracy: 0.2218
Validation set:	Average loss: 2.2492, Accuracy: 0.2488

Epoch 6: training...
Train set:	Average loss: 2.2380, Accuracy: 0.2241
Validation set:	Average loss: 2.2406, Accuracy: 0.2040

Epoch 7: training...
Train set:	Average loss: 2.2404, Accuracy: 0.2159
Validation set:	Average loss: 2.2397, A

## Part 4

In this part, you will use PyTorch to implement and train a multi-layer fully-connected neural network to classify MNIST digits.

Your network must have three hidden layers with 128, 64, and 32 hidden units respectively.

The same restrictions as in Part 3 apply.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultilayerClassifier` class below by filling in the missing parts of the following methods (expected behaviour is prescribed in the documentation):

    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 97% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.

3. Describe an alternative strategy for initializing weights that may perform better than the strategy we have used here.

In [599]:
# *CODE FOR PART 4.1 IN THIS CELL*

class MultilayerClassifier:
    def __init__(self, activation_fun="sigmoid", weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        super().__init__()
        self.activation_fun = activation_fun
        self.weight_init_sd = weight_init_sd

        if self.activation_fun == "relu":
            self.activation = F.relu
        elif self.activation_fun == "sigmoid":
            self.activation = torch.sigmoid
        elif self.activation_fun == "tanh":
            self.activation = torch.tanh
        else:
            raise NotImplementedError()

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        self.variables = 28*28
        self.outcomes = 10
        
        m = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([self.weight_init_sd]))
        
        self.p1 = m.sample(torch.Size([self.variables, 128])).view([self.variables, 128]) 
        self.p1.requires_grad_ = True
        self.b1 = torch.ones((1, 128))
        self.b1.requires_grad_ = True
        
        self.p2 = m.sample(torch.Size([128, 64])).view([128, 64]) 
        self.p2.requires_grad_ = True
        self.b2 = torch.ones((1, 64))
        self.b2.requires_grad_ = True
        
        self.p3 = m.sample(torch.Size([64, 32])).view([64, 32]) 
        self.p3.requires_grad_ = True
        self.b3 = torch.ones((1, 32))
        self.b3.requires_grad_ = True
        
        self.p4 = m.sample(torch.Size([32, self.outcomes])).view([32, self.outcomes]) 
        self.p4.requires_grad_ = True
        self.b4 = torch.ones((1, self.outcomes))
        self.b4.requires_grad_ = True
        
        self.params = torch.tensor([self.p1, self.b1, self.p2, self.b2, self.p3, self.b3, self.p4, self.b4])
        self.params.requires_grad_()
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        flat_inputs = inputs.view(- 1, self.variables)
        batch_size = flat_inputs.shape[0]
        flat_bias = torch.cat((flat_inputs, torch.ones(batch_size, 1)), 1)
        
        # Neural network
        h = self.activation(flat_bias)
        return self.activation(h)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return [self.p1, self.b1, self.p2, self.b2, self.p3, self.b3, self.p4, self.b4]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        l1_params = abs(self.params).sum()
        return l1_params
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        params_prod = self.params.matmul(self.params.t())
        return torch.diag(params_prod).sum() 
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [600]:
# *CODE FOR PART 4.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *

model = MultilayerClassifier(activation_fun='sigmoid', weight_init_sd=1.0)
res = run_experiment(
    model,
    optimizer=optim.SGD(model.parameters(), 0.1),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=3,
    l1_penalty_coef=1.0,
    l2_penalty_coef=1.0,
    suppress_output=False
)

ValueError: only one element tensors can be converted to Python scalars

\**ANSWERS FOR PART 4.3 IN THIS CELL*\*